In [1]:
!pip install geopy tqdm

In [2]:
import pandas as pd
import requests
import os
from datetime import datetime, timedelta
from geopy.distance import geodesic
from tqdm import tqdm
import logging

In [3]:
# Konfiguracja stałych - położenia Warszawy oraz nazw plików
LAT_WAW = 52.159499362
LONG_WAW = 20.966996132
DATE_URL_PART = "2025/12/01" # Format do URL ADSBExchange
DATE_FILENAME = "20251201"   # Format do nazwy pliku lokalnego

# Ustawienie ścieżek
#base_dir = os.path.dirname(os.path.abspath(__file__)) #Dla .py
base_dir = os.getcwd() #Dla jupytera
arrivals_path = os.path.join(base_dir, '..', 'data', f'ax_arrivals_{DATE_FILENAME}.csv')
output_path = os.path.join(base_dir, '..', 'data', f'loty_waw_8_23_{DATE_FILENAME}.csv')

In [4]:
#funkcje pomocnicze
def download_json_to_pandas(url):
    """Pobiera JSON z ADSBExchange i zamienia na DataFrame"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        # Dane o samolotach są w kluczu 'aircraft' lub 'ac' zależnie od wersji
        aircraft_data = json_data.get('aircraft', []) or json_data.get('ac', [])
        return pd.DataFrame(aircraft_data)
    except Exception as e:
        # Czasami pojedynczy plik może nie istnieć lub zerwać połączenie
        return pd.DataFrame()

def haversine_distance(row):
    """Oblicza dystans do Okęcia (wymaga kolumn 'lat', 'lon')"""
    if pd.isna(row['lat']) or pd.isna(row['lon']):
        return None
    return geodesic((row['lat'], row['lon']), (LAT_WAW, LONG_WAW)).kilometers

In [5]:
# Lista kolumn do zachowania
COLS_TO_KEEP = [
    'hex', 'flight', 'category', 'lat', 'lon', 
    'alt_baro', 'alt_geom', 'gs', 'track', 
    'geom_rate', 'track_rate', 'seen_pos',
    'snapshot_time', 'distance_km'
]

In [6]:
print("Wczytywanie listy przylotów/odlotów (White-list)...")

try:
    df_arrivals = pd.read_csv(arrivals_path, low_memory=False)
    
    # Normalizacja callsign (usuwamy spacje)
    if 'callsign' in df_arrivals.columns:
        df_arrivals['callsign'] = df_arrivals['callsign'].astype(str).str.strip().str.upper()
    elif 'flight' in df_arrivals.columns:
        df_arrivals['callsign'] = df_arrivals['flight'].astype(str).str.strip().str.upper()
        
    # FILTRACJA: Tylko loty powiązane z Warszawą (EPWA)
    # nie filtrujemy tu czasu, bierzemy cały dzień,
    # żeby nie przegapić opóźnionych lotów.
    mask_waw = (df_arrivals['dest'] == "EPWA") | (df_arrivals['orig'] == "EPWA")
    df_waw = df_arrivals[mask_waw].copy()
    
    # Lista unikalnych znaków wywoławczych (np. LOT123)
    unique_flights_to_waw = df_waw['callsign'].unique().tolist()
    
    print(f"Znaleziono {len(unique_flights_to_waw)} unikalnych lotów powiązanych z EPWA w pliku planowym.")

except FileNotFoundError:
    print(f"BŁĄD: Nie znaleziono pliku {arrivals_path}")
    exit()

print("Generowanie listy czasów do pobrania (08:00 - 10:00)...")

start_time = datetime.strptime("080000", "%H%M%S")
end_time = datetime.strptime("230000", "%H%M%S")
time_intervals = []

current = start_time
while current <= end_time:
    time_intervals.append(current.strftime("%H%M%S"))
    current += timedelta(seconds=5) # Pobieranie co 5 sekund

print(f"Liczba snapshotów do pobrania: {len(time_intervals)}")


Wczytywanie listy przylotów/odlotów (White-list)...
Znaleziono 459 unikalnych lotów powiązanych z EPWA w pliku planowym.
Generowanie listy czasów do pobrania (08:00 - 10:00)...
Liczba snapshotów do pobrania: 10801


In [7]:
# Kontener na wszystkie pobrane dane
all_data_frames = []
missed_downloads = []

In [9]:
from concurrent.futures import ThreadPoolExecutor

# Tworzymy sesję, aby reużywać połączenia TCP (bardzo ważne dla szybkości!)
session = requests.Session()

def download_and_process(time_str):
    """Funkcja pomocnicza do przetwarzania jednego pliku w osobnym wątku"""
    url = f"https://samples.adsbexchange.com/readsb-hist/{DATE_URL_PART}/{time_str}Z.json.gz"
    
    try:
        # Pobieranie danych przez sesję
        response = session.get(url, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        aircraft_data = json_data.get('aircraft', []) or json_data.get('ac', [])
        df_snap = pd.DataFrame(aircraft_data)
    except Exception:
        return None

    if not df_snap.empty and 'flight' in df_snap.columns:
        # Standaryzacja i filtrowanie (logika z Twojego oryginalnego kodu)
        df_snap['flight'] = df_snap['flight'].astype(str).str.strip().str.upper()
        df_snap = df_snap[df_snap['flight'].isin(unique_flights_to_waw)].copy()
        
        if not df_snap.empty:
            df_snap['snapshot_time'] = time_str
            df_snap = df_snap.dropna(subset=['lat', 'lon'])
            
            if not df_snap.empty:
                # Obliczenia dystansu (haversine_distance z komórki nr 4)
                df_snap['distance_km'] = df_snap.apply(haversine_distance, axis=1)
                
                # Wybór kolumn (COLS_TO_KEEP z komórki nr 5)
                existing_cols = [c for c in COLS_TO_KEEP if c in df_snap.columns]
                return df_snap[existing_cols]
    return None

print(f"Rozpoczęcie równoległego pobierania dla {len(time_intervals)} snapshotów...")

# max_workers=20 oznacza 20 równoległych zapytań. 
# Jeśli masz bardzo szybki internet, możesz spróbować wartości 30 lub 40.
with ThreadPoolExecutor(max_workers=40) as executor:
    # executor.map rozdziela zadania na wątki
    results = list(tqdm(executor.map(download_and_process, time_intervals), total=len(time_intervals)))

# Filtrujemy wyniki (usuwamy None) i przypisujemy do Twojej listy
all_data_frames = [df for df in results if df is not None]

print(f"Zakończono! Zebrano dane z {len(all_data_frames)} snapshotów.")


Rozpoczęcie równoległego pobierania dla 10801 snapshotów...


100%|██████████████████████████████████████████████████████████████████████████| 10801/10801 [1:07:21<00:00,  2.67it/s]


Zakończono! Zebrano dane z 10800 snapshotów.


In [10]:
# Zapis wyników

if all_data_frames:
    print("Łączenie danych i zapisywanie...")
    master_df = pd.concat(all_data_frames, ignore_index=True)
    
    # Zapis do pliku w folderze data
    master_df.to_csv(output_path, index=False)
    print(f"SUKCES! Zapisano {len(master_df)} punktów pomiarowych do pliku:")
    print(output_path)
    
    print(master_df[['snapshot_time', 'flight', 'lat', 'lon', 'distance_km', 'alt_baro']].head())
else:
    print("Ostrzeżenie: Nie pobrano żadnych danych pasujących do kryteriów.")

Łączenie danych i zapisywanie...
SUKCES! Zapisano 547822 punktów pomiarowych do pliku:
C:\Users\Michał J\Desktop\MJ_mgr\Codes\..\data\loty_waw_8_23_20251201.csv
  snapshot_time   flight        lat        lon  distance_km alt_baro
0        080000   LOT16M  58.500000 -28.533300  3143.072981    37000
1        080000   LOT3PK  56.017332 -11.097479  2121.332828    39000
2        080000  TAP120Y  41.782589  -5.650997  2311.731328    36025
3        080000   WZZ4LM  53.334709  -2.862959  1606.980074   ground
4        080000  WZZ20CS  51.880793  -0.377312  1460.261310   ground
